## SAMBERT个性化语音合成模型介绍
本文将介绍SAMBERT个性化语音合成模型, 包括其模型结构, 如何在ModelScope上体验模型效果, 以及如何使用开源代码训练SAMBERT个性化语音合成模型.

### 个性化语音合成

语音合成(Text-to-Speech, TTS) 是指将输入文字合成为对应语音信号的功能，即赋予计算机“说”的能力，是人机交互中重要的一环。现代语音合成最早可以追溯到1939年贝尔实验室制造的第一个电子语音合成器，后来历经共振峰合成、PSOLA合成、Unit Selection波形拼接、统计参数合成几代的发展，在2016年随着WaveNet的出现步入了深度学习合成时代，此时语音合成的效果已经表现出了比拟真人的水准。

提到个性化定制语音，大家并不陌生，许多平台会选择一些大家耳熟能详的明星，进行声音定制，并普遍应用在语音导航，文字播报，小说阅读等场景中。这项技术来自文本到语音的服务，一般来说，语音合成数据需要专业播音员在录音棚录制，且数据量都是以500-1000句话起步，这种标准定制的流程，无论是对播音员、录制条件、录制数量和成本都提出了较高的要求。

Personal TTS，即个性化语音合成，是通过身边的一些常见录音设备（手机、电脑、录音笔等），录取目标说话人的少量语音片段后，构建出这个人的语音合成系统。相比于标准定制，个性化定制的技术难点在于，数据量有限（20句话）、数据质量不佳和流程全自动化。而它的意义在于进一步降低语音合成的定制门槛，能够将语音合成定制推广到普通用户。

学术界有很多关于声音克隆的工作，论文陈述效果很好。考虑到落地应用场景的效果，达摩院以自研语音合成系统KAN-TTS的迁移学习能力为基础，设计了一套较为完善的个性化语音合成方案。用户只需要录制20句话，经过几分钟的训练，就能够获得一个较好的个性化声音。

<img src="./asserts/sambert01.png" style="margin-left: 0px" width="800px">

### 模型框架
模型框架主要由三个部分组成：

1. 数据自动化处理和标注
2. 韵律建模SAMBERT声学模型
3. 基于说话人特征信息的个性化语音合成

<img src="./asserts/sambert02.png" style="margin-left: 0px" width="800px">

### 数据自动化处理和标注

在用户录制完音频之后，我们只有<文本，音频>，而语音合成是需要一些额外的标注信息：韵律标注、音素时长标注。为了获得较好的标注信息，我们采用了一种融合了多种原子能力的全自动化处理和标注流程，包括，韵律预测、ASR、VAD和语音增强等。通过测试集测试，该自动化流程产生的标注信息，在准确度能够满足个性化的需求。

此外，我们也在modelscope上发布了TTS-Autolabel自动化数据标注工具，旨在降低TTS数据标注门槛，使开发者更便捷的定制个性化语音合成模型，具体使用方式及相关教程请进入TTS-AutoLabel工具主页获取。

### 韵律建模SAMBERT声学模型

在整个链路中，和效果最相关的模块就是声学模型。在语音合成领域，类似FastSpeech的Parallel模型是目前的主流，它针对基频（pitch）、能量（energy）和时长（duration）三种韵律表征分别建模。但是，该类模型普遍存在一些效果和性能上的问题，例如，独立建模时长、基频、能量，忽视了其内在联系；完全非自回归的网络结构，无法满足工业级实时合成需求；帧级别基频和能量预测不稳定。 因此达摩院语音实验室设计了SAMBERT，一种基于Parallel结构的改良版TTS模型，它具有以下优点：

1. Backbone采用Self-Attention-Mechanism(SAM)，提升模型建模能力。
2. Encoder部分采用BERT进行初始化，引入更多文本信息，提升合成韵律。
3. Variance Adaptor对音素级别的韵律(基频、能量、时长)轮廓进行粗粒度的预测，再通过decoder进行帧级别细粒度的建模;并在时长预测时考虑到其与基频、能量的关联信息，结合自回归结构，进一步提升韵律自然度.
4. Decoder部分采用PNCA AR-Decoder[@li2020robutrans]，自然支持流式合成。

<img src="./asserts/sambert03.jpeg" style="margin-left: 0px" width="800px">

### 基于说话人特征信息的个性化语音合成

如果需要进行迁移学习，那么需要先构建多说话人的声学模型，不同说话人是通过可训练的说话人编码（speaker embedding）进行区分的。给定新的一个说话人，一般通过随机初始化一个speaker embedding，然后再基于这个说话人的数据进行更新（见下图说话人空间1）。对于个性化语音合成来说，发音人的数据量比较少，学习难度很大，最终合成声音的相似度就无法保证。因此，我们采用说话人特征信息来表示每个说话人，此时，以少量说话人数据初始化的 speaker embedding 距离实际的目标说话人更近得多（见下图说话人空间2），学习难度小，此时合成声音的相似度就比较高。采用基于说话人特征信息的个性化语音合成，使得在20句条件下，依旧能够有较好的相似度。

<img src="./asserts/sambert04.png" style="margin-left: 0px" width="800px">

## 代码实践

### 1. 训练数据准备


- test_female 文件夹

### 2. 前置工作

安装最新版tts-autolabel, 在Notebook中新建一个代码块，输入如下代码并运行



In [1]:
# 运行此代码块安装tts-autolabel
import sys
!{sys.executable} -m pip install -U tts-autolabel -f https://modelscope.oss-cn-beijing.aliyuncs.com/releases/repo.html

Looking in indexes: https://mirrors.aliyun.com/pypi/simple
Looking in links: https://modelscope.oss-cn-beijing.aliyuncs.com/releases/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.3/117.3 kB 1.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 11.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.0/209.0 kB 13.9 MB/s eta 0:00:00
DEPRECATION: pytorch-lightning 1.7.7 has a non-standard dependency specifier torch>=1.9.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.3
    Uninstalling numpy-1.26.3:
      Successfull

In [ ]:
# 如果由于网络问题安装失败，可以尝试使用国内镜像源, 在Notebook中新建一个代码块，输入如下代码并运行
# !{sys.executable} -m pip config set global.index-url https://mirrors.aliyun.com/pypi/simple/

### 3. 运行TTS-AutoLabel自动标注

首先在Notebook左侧新建TTS-AutoLabel工作文件夹output_training_data, 然后在Notebook code block中输入如下代码并运行, 由于Notebook运行环境已经安装了modelscope和tts-autolabel此处可以直接导入。


二十条音频标注时间大约30秒。标注完成后在output_training_data中可以获取音频对应的TTS训练数据。

In [ ]:
!mkdir output_training_data

In [2]:
from modelscope.tools import run_auto_label

input_wav = "./data/test_female/"
output_data = "./data/output_training_data/"

ret, report = run_auto_label(input_wav=input_wav, work_dir=output_data, resource_revision="v1.0.7")

2024-03-11 20:12:56,920 - modelscope - INFO - PyTorch version 2.1.2+cu121 Found.
2024-03-11 20:12:56,922 - modelscope - INFO - TensorFlow version 2.14.0 Found.
2024-03-11 20:12:56,923 - modelscope - INFO - Loading ast index from /mnt/workspace/.cache/modelscope/ast_indexer
2024-03-11 20:12:56,923 - modelscope - INFO - No valid ast index found from /mnt/workspace/.cache/modelscope/ast_indexer, generating ast index from prebuilt!
2024-03-11 20:12:57,247 - modelscope - INFO - Loading done! Current index file version is 1.12.0, with md5 509123dba36c5e70a95f6780df348471 and a total number of 964 components indexed
/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-03-11 20:12:59,818 - modelscope - INFO - Use user-specified model revision: v1.0.7
Downloading: 100%|██████████| 185/185 [00:00<00:00

text.cc: festival_Text_init


100%|██████████| 20/20 [00:02<00:00,  8.82it/s]
INFO:root:pre-break recording in paragraph by vad.
INFO:root:Generate phone interval by fa align.
INFO:root:prosody_dir=/mnt/workspace/output_training_data/paragraph/prosody
INFO:root:FA processing...
INFO:root:---  New folder /mnt/workspace/output_training_data/raw_ali...  ---
INFO:root:---  OK  ---
INFO:root:---  New folder /mnt/workspace/output_training_data/raw_interval...  ---
INFO:root:---  OK  ---
  0%|          | 0/20 [00:00<?, ?it/s]INFO:root:wav: /mnt/workspace/output_training_data/wav_cut_16k/SSB00180012_S0000.wav, text: 我 怎么 会 和你 扯上 关系 
avr_tok:53, max_tok:109, avr_wbtok:2, max_wbtok:4, avr_wetok:3, max_wetok:6, avr_we_arc 6 avr_lmcall:6 p6, avr_histbeam:150.000000, max_histbeam:150.000000, avr_wbhistbeam:100.000000, max_wbhistbeam:100.000000
INFO:root:wav: /mnt/workspace/output_training_data/wav_cut_16k/SSB00180288_S0000.wav, text: 我就 帮你 逃命 
avr_tok:22, max_tok:34, avr_wbtok:1, max_wbtok:3, avr_wetok:1, max_wetok:3, avr_we_ar

### 4. 基于PTTS-basemodel微调

获得标注好的训练数据后，我们进行模型微调，新建一个Notebook代码块，并输入如下代码运行

微调过程大概需要5分钟的时间，请耐心等待。

In [ ]:
!mkdir pretrain_work_dir

In [4]:
from modelscope.metainfo import Trainers
from modelscope.trainers import build_trainer
from modelscope.utils.audio.audio_utils import TtsTrainType

pretrained_model_id = 'damo/speech_personal_sambert-hifigan_nsf_tts_zh-cn_pretrain_16k'

dataset_id = "./data/output_training_data/"
pretrain_work_dir = "./data/pretrain_work_dir/"
        
# 训练信息，用于指定需要训练哪个或哪些模型，这里展示AM和Vocoder模型皆进行训练
# 目前支持训练：TtsTrainType.TRAIN_TYPE_SAMBERT, TtsTrainType.TRAIN_TYPE_VOC
# 训练SAMBERT会以模型最新step作为基础进行finetune
train_info = {
    TtsTrainType.TRAIN_TYPE_SAMBERT: {  # 配置训练AM（sambert）模型
        'train_steps': 202,               # 训练多少个step 
        'save_interval_steps': 200,       # 每训练多少个step保存一次checkpoint
        'log_interval': 10               # 每训练多少个step打印一次训练日志
    }
}

# 配置训练参数，指定数据集，临时工作目录和train_info
kwargs = dict(
    model=pretrained_model_id,                  # 指定要finetune的模型
    model_revision = "v1.0.6",
    work_dir=pretrain_work_dir,                 # 指定临时工作目录
    train_dataset=dataset_id,                   # 指定数据集id
    train_type=train_info                       # 指定要训练类型及参数
)

trainer = build_trainer(Trainers.speech_kantts_trainer,
                        default_args=kwargs)

trainer.train()

2024-03-11 20:14:40,830 - modelscope - INFO - Use user-specified model revision: v1.0.6
2024-03-11 20:14:42,012 - modelscope - INFO - Use user-specified model revision: v1.0.6
2024-03-11 20:14:42,636 - modelscope - INFO - Set workdir to ./pretrain_work_dir/
2024-03-11 20:14:42,638 - modelscope - INFO - load ./output_training_data/
2024-03-11 20:14:42,639 - modelscope - WARNING - Download interval is too small, use local cache
2024-03-11 20:14:43,419 - modelscope - INFO - am_config=./pretrain_work_dir/orig_model/basemodel_16k/sambert/config.yaml voc_config=./pretrain_work_dir/orig_model/basemodel_16k/hifigan/config.yaml
2024-03-11 20:14:43,420 - modelscope - INFO - audio_config=./pretrain_work_dir/orig_model/basemodel_16k/audio_config_se_16k.yaml
2024-03-11 20:14:43,420 - modelscope - INFO - am_ckpts=OrderedDict([(2400000, './pretrain_work_dir/orig_model/basemodel_16k/sambert/ckpt/checkpoint_2400000.pth')])
2024-03-11 20:14:43,421 - modelscope - INFO - voc_ckpts=OrderedDict([(2400000, '

Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed


100%|██████████| 40/40 [00:00<00:00, 16376.00it/s]
INFO:root:TextScriptConvertor.process:
Save script to: ./pretrain_work_dir/data/Script.xml
INFO:root:TextScriptConvertor.process:
Save metafile to: ./pretrain_work_dir/data/raw_metafile.txt
INFO:root:[AudioProcessor] Initialize AudioProcessor.
INFO:root:[AudioProcessor] config params:
INFO:root:[AudioProcessor] wav_normalize: True
INFO:root:[AudioProcessor] trim_silence: True
INFO:root:[AudioProcessor] trim_silence_threshold_db: 60
INFO:root:[AudioProcessor] preemphasize: False
INFO:root:[AudioProcessor] sampling_rate: 16000
INFO:root:[AudioProcessor] hop_length: 200
INFO:root:[AudioProcessor] win_length: 1000
INFO:root:[AudioProcessor] n_fft: 2048
INFO:root:[AudioProcessor] n_mels: 80
INFO:root:[AudioProcessor] fmin: 0.0
INFO:root:[AudioProcessor] fmax: 8000.0
INFO:root:[AudioProcessor] phone_level_feature: True
INFO:root:[AudioProcessor] se_feature: True
INFO:root:[AudioProcessor] norm_type: mean_std
INFO:root:[AudioProcessor] max_no

### 5. 体验模型合成效果

使用上一步微调得到的模型合成音频，新建一个Notebook代码块, 输入以下代码

In [5]:
import os
from modelscope.models.audio.tts import SambertHifigan
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks

model_dir = os.path.abspath("./data/pretrain_work_dir")

custom_infer_abs = {
    'voice_name':
    'F7',
    'am_ckpt':
    os.path.join(model_dir, 'tmp_am', 'ckpt'),
    'am_config':
    os.path.join(model_dir, 'tmp_am', 'config.yaml'),
    'voc_ckpt':
    os.path.join(model_dir, 'orig_model', 'basemodel_16k', 'hifigan', 'ckpt'),
    'voc_config':
    os.path.join(model_dir, 'orig_model', 'basemodel_16k', 'hifigan',
             'config.yaml'),
    'audio_config':
    os.path.join(model_dir, 'data', 'audio_config.yaml'),
    'se_file':
    os.path.join(model_dir, 'data', 'se', 'se.npy')
}
kwargs = {'custom_ckpt': custom_infer_abs}

model_id = SambertHifigan(os.path.join(model_dir, "orig_model"), **kwargs)

inference = pipeline(task=Tasks.text_to_speech, model=model_id)
output = inference(input="今天的天气真不错")

import IPython.display as ipd
ipd.Audio(output["output_wav"], rate=16000)

2024-03-11 20:17:35,872 - modelscope - INFO - am_config=/mnt/workspace/pretrain_work_dir/tmp_am/config.yaml voc_config=/mnt/workspace/pretrain_work_dir/orig_model/basemodel_16k/hifigan/config.yaml
2024-03-11 20:17:35,872 - modelscope - INFO - audio_config=/mnt/workspace/pretrain_work_dir/data/audio_config.yaml
2024-03-11 20:17:35,873 - modelscope - INFO - am_ckpts=OrderedDict([(2400000, '/mnt/workspace/pretrain_work_dir/tmp_am/ckpt/checkpoint_2400000.pth'), (2400200, '/mnt/workspace/pretrain_work_dir/tmp_am/ckpt/checkpoint_2400200.pth')])
2024-03-11 20:17:35,873 - modelscope - INFO - voc_ckpts=OrderedDict([(2400000, '/mnt/workspace/pretrain_work_dir/orig_model/basemodel_16k/hifigan/ckpt/checkpoint_2400000.pth')])
2024-03-11 20:17:35,874 - modelscope - INFO - se_path=/mnt/workspace/pretrain_work_dir/data/se/se.npy se_model_path=/mnt/workspace/pretrain_work_dir/orig_model/se.onnx
2024-03-11 20:17:35,874 - modelscope - INFO - mvn_path=/mnt/workspace/pretrain_work_dir/orig_model/mvn.npy
fe

Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed


/opt/conda/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


Removing weight norm...


点击播放控件即可体验合成音频。

### Demo 合成音频

In [9]:
text_list = [
    
'一条鲜红的鱼优雅地游过,它的鳞片在水中闪闪发光。',
'从另一边,一条翠绿的鱼也加入进来,在水中自在地游动着。',
'第三条金黄色的鱼出现了,它的身体在阳光下熠熠生辉。',
'最后,一条漆黑的鱼也悠闲地游了过来,加入了这个多彩的鱼群。',
'红绿黄黑四条鱼在水中嬉戏玩耍,互相追逐,场面十分热闹有趣。',
'这时,一大群游客来到了水族馆,对面前的景象啧啧称奇。',
'游客们兴奋地拍照录像,捕捉这独一无二的场景。',
'突然，红色的那条鱼跳跃着，远远地向一名小女孩挥动鱼尾巴致意。',
'小女孩开心极了,向红色的小鱼挥手回应,两者有了一段有趣的互动。',
'整个水族馆里洋溢着快乐的氛围,游客和鱼儿共同创造了一个独特而难忘的时刻。',
    
]
            

In [11]:
# 文本列表
# text_list = ["今天的天气真不错", "明天会下雨吗?", "我想吃pizza", "快点睡觉吧"]

model_dir = os.path.abspath("./data/audio/audio_output7")

# 遍历文本列表,生成并保存音频文件
for i, text in enumerate(text_list):
    output = inference(input=text)
    audio_file = f"{model_dir}/output_{i}.wav"
    with open(audio_file, "wb") as f:
        f.write(output["output_wav"])
    print(f"已保存音频文件: {audio_file}")

已保存音频文件: /mnt/workspace/audio_output/output_0.wav
已保存音频文件: /mnt/workspace/audio_output/output_1.wav
已保存音频文件: /mnt/workspace/audio_output/output_2.wav
已保存音频文件: /mnt/workspace/audio_output/output_3.wav
已保存音频文件: /mnt/workspace/audio_output/output_4.wav
已保存音频文件: /mnt/workspace/audio_output/output_5.wav
已保存音频文件: /mnt/workspace/audio_output/output_6.wav
已保存音频文件: /mnt/workspace/audio_output/output_7.wav
已保存音频文件: /mnt/workspace/audio_output/output_8.wav
已保存音频文件: /mnt/workspace/audio_output/output_9.wav


In [12]:
text_list = [
    
'阳光透过树叶的缝隙洒在波光粼粼的水面上，一条红色的鱼悠然游过，留下一串波纹。',
'一条绿色的鱼从水草丛中游出，随着它的动作，水草轻轻摇曳，阳光照射下显得格外生动。',
'随即，一条黄色的鱼从水底快速游过，其黄色鳞片在光线下闪耀着金黄色的光芒。',
'紧接着，一条神秘的黑色鱼悄无声息地出现，它的身影在水中忽隐忽现，仿佛守护着这片秘境。',
'红色、绿色、黄色和黑色的鱼开始环绕游动，它们似乎在进行一场优雅的水下舞蹈。',
'此时，一群游客看着五颜六色的鱼群，手持相机和手机，兴奋地记录下这难得一见的景象。',
'其中一名小女孩蹲下身子，伸手试图触摸正在水面嬉戏的鱼，脸上露出开心的笑容。',
'突然，红色的鱼跃出水面，好像在回应那位游客，空中完成一个优雅的弧线，又轻巧落回水中。',
'其他游客见状纷纷欢呼起来，有的还尝试模仿那位互动游客的动作，希望能引起鱼儿的注意。',
'整个湖面和观看的人群，从大地到天空，都是一片欢乐和谐的气氛。',
    
]
            

In [13]:
# 文本列表
# text_list = ["今天的天气真不错", "明天会下雨吗?", "我想吃pizza", "快点睡觉吧"]

model_dir = os.path.abspath("./data/audio/audio_output9")

# 遍历文本列表,生成并保存音频文件
for i, text in enumerate(text_list):
    output = inference(input=text)
    audio_file = f"{model_dir}/output_{i}.wav"
    with open(audio_file, "wb") as f:
        f.write(output["output_wav"])
    print(f"已保存音频文件: {audio_file}")

已保存音频文件: /mnt/workspace/audio_output9/output_0.wav
已保存音频文件: /mnt/workspace/audio_output9/output_1.wav
已保存音频文件: /mnt/workspace/audio_output9/output_2.wav
已保存音频文件: /mnt/workspace/audio_output9/output_3.wav
已保存音频文件: /mnt/workspace/audio_output9/output_4.wav
已保存音频文件: /mnt/workspace/audio_output9/output_5.wav
已保存音频文件: /mnt/workspace/audio_output9/output_6.wav
已保存音频文件: /mnt/workspace/audio_output9/output_7.wav
已保存音频文件: /mnt/workspace/audio_output9/output_8.wav
已保存音频文件: /mnt/workspace/audio_output9/output_9.wav
